In [ ]:
from typing import Dict, List, Tuple
from collections import namedtuple
import numpy as np
from research.weight_estimation.keypoint_utils.body_parts import core_body_parts
from research.weight_estimation.keypoint_utils.keypoint_transformations import get_raw_3d_coordinates
import datetime as dt
import time
import json
import os

from research.utils.data_access_utils import RDSAccessUtils

In [ ]:
rds = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

In [ ]:
query = """
    select * from prod.biomass_computations
    where pen_id=88
    and captured_at >= '2020-02-10'
    and captured_at <= '2020-02-20'
    and akpd_score >= 0.0
"""

df = rds.extract_from_database(query)

In [ ]:
FishDetection = namedtuple('FishDetection', ['left_crop_url', 'right_crop_url', 'captured_at',
                                             'annotation', 'camera_metadata', 'estimated_weight_g',
                                             'estimated_k_factor', 'akpd_score'])


def get_left_right_keypoint_arrs(annotation: Dict[str, List[Dict]]) -> Tuple:
    """Gets numpy array of left and right keypoints given input keypoint annotation.
    Args:
        annotation: dict with keys 'leftCrop' and 'rightCrop'. Values are lists where each element
        is a dict with keys 'keypointType', 'xCrop' (num pixels from crop left edge),
        'yCrop' (num pixels from crop top edge), 'xFrame' (num pixels from full frame left edge),
        and 'yFrame' (num pixels from full frame top edge).
    Returns:
        X_left: numpy array containing left crop (xFrame, yFrame) for each key-point ordered
        alphabetically.
        X_right: same as above, but for right crop.
    """

    left_keypoints, right_keypoints = {}, {}
    for item in annotation['leftCrop']:
        body_part = item['keypointType']
        left_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    for item in annotation['rightCrop']:
        body_part = item['keypointType']
        right_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    left_keypoint_arr, right_keypoint_arr = [], []
    for body_part in core_body_parts:
        left_keypoint_arr.append(left_keypoints[body_part])
        right_keypoint_arr.append(right_keypoints[body_part])

    X_left = np.array(left_keypoint_arr)
    X_right = np.array(right_keypoint_arr)
    return X_left, X_right


def ts_to_unix_epoch(ts):
    epoch = time.mktime(ts.timetuple())
    return epoch

def time_gap_between_fds(fd, hist_fd):
    time_gap = ts_to_unix_epoch(fd.captured_at) - ts_to_unix_epoch(hist_fd.captured_at)
    return time_gap


def check_if_consecutive_match(fd: FishDetection, hist_fd: FishDetection,
                               lookback_period_s: float, pixel_threshold: int = 300) -> bool:

#     print(fd.captured_at, hist_fd.captured_at, ts_to_unix_epoch(fd.captured_at), time_gap)
    time_gap = time_gap_between_fds(fd, hist_fd)
    if time_gap == 0 or time_gap > lookback_period_s:
        return False
    
    X_left, X_right = get_left_right_keypoint_arrs(fd.annotation)
    X_left_hist, X_right_hist = get_left_right_keypoint_arrs(hist_fd.annotation)

    # perform reasonable shift check
    eye, tail = core_body_parts.index('EYE'), core_body_parts.index('TAIL_NOTCH')
    eye_translation = X_left[eye] - X_left_hist[eye]
    tail_translation = X_left[tail] - X_left_hist[tail]
    if ts_to_unix_epoch(fd.captured_at) == 1597203250.0:
        print(np.linalg.norm(eye_translation - tail_translation))
    if np.linalg.norm(eye_translation - tail_translation) > pixel_threshold:
        return False

    # perform forward movement check
    fish_displacement = X_left[eye] - X_left[tail]
    angle_difference = np.arccos(np.dot(eye_translation, fish_displacement) / (np.linalg.norm(eye_translation) * np.linalg.norm(fish_displacement)))
    if np.abs(angle_difference) > np.pi / 4.0:
        return False
    if np.linalg.norm(eye_translation) > 2 * np.linalg.norm(fish_displacement):
        return False
    
#     forward_movement_check = np.sign(X_left[eye][0] - X_left[tail][0]) == \
#                              np.sign(X_left_hist[eye][0] - X_left_hist[tail][0]) == \
#                              np.sign(X_left[eye][0] - X_left_hist[eye][0])

    return True


def compute_speed(fd: FishDetection, hist_fd: FishDetection) -> float:
    X_world = get_raw_3d_coordinates(fd.annotation, fd.camera_metadata)
    X_world_hist = get_raw_3d_coordinates(hist_fd.annotation, hist_fd.camera_metadata)
    distance = np.median(np.linalg.norm(X_world - X_world_hist, axis=0))
    time = ts_to_unix_epoch(fd.captured_at) - ts_to_unix_epoch(hist_fd.captured_at)
    return float(distance) / time


def compute_swimming_speeds(fish_detections: List[FishDetection],
                           lookback_period_s: float = 5.0) -> List:

    historical_fds = []
    speed_objects = []
    for fd in sorted(fish_detections, key=lambda x: x.captured_at):
        if fd.annotation is None:
            continue
        if 'leftCrop' not in fd.annotation or 'rightCrop' not in fd.annotation:
            continue
        
        # purge historical fds
        historical_fds = [hist_fd for hist_fd in historical_fds if time_gap_between_fds(fd, hist_fd) < lookback_period_s]
        
        for hist_fd in historical_fds:
            is_consecutive_match = check_if_consecutive_match(fd, hist_fd, lookback_period_s)
            if is_consecutive_match:
                speed = compute_speed(fd, hist_fd)
                if speed < 1.0:
                    speed_obj = (fd, hist_fd, hist_fd.captured_at, speed, hist_fd.left_crop_url, 
                                 fd.left_crop_url, hist_fd.estimated_weight_g, fd.estimated_weight_g,
                                 hist_fd.akpd_score, fd.akpd_score)
                    speed_objects.append(speed_obj)
        historical_fds.append(fd)

    return speed_objects


In [ ]:
fish_detections = []
for idx, row in df.iterrows():
    fd = FishDetection(
        left_crop_url=row.left_crop_url,
        right_crop_url=row.right_crop_url,
        captured_at=row.captured_at,
        annotation=row.annotation,
        camera_metadata=row.camera_metadata,
        estimated_weight_g=row.estimated_weight_g,
        estimated_k_factor=row.estimated_k_factor,
        akpd_score=row.akpd_score
    )
    fish_detections.append(fd)

In [ ]:
fd1 = dict(speed_objects[0][0]._asdict())
fd2 = dict(speed_objects[0][1]._asdict())
fd1['captured_at'] = str(fd1['captured_at'])
fd2['captured_at'] = str(fd2['captured_at'])

print(json.dumps([fd1, fd2], indent=4))

In [ ]:
str(fish_detections[0].captured_at)

In [ ]:
import datetime as dt
import time

In [ ]:
time.mktime(dt.datetime.strptime(str(fish_detections[0].captured_at).split('+')[0], '%Y-%m-%d %H:%M:%S.%f').timetuple())

In [ ]:
speed_objects = compute_swimming_speeds(fish_detections)

In [ ]:
x = np.array([(speed_obj[4], speed_obj[5], speed_obj[6], speed_obj[7]) for speed_obj in speed_objects])

In [ ]:
np.median(np.abs((x[:, 0] - x[:, 1]) / x[:, 1]))

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(20, 10))
plt.hist([speed_obj[1] for speed_obj in speed_objects], bins=50)
plt.grid()
plt.show()

In [ ]:
import pandas as pd


In [ ]:
swimming_speed_df = pd.DataFrame({
    'captured_at': [so[0] for so in speed_objects],
    'speed': [so[1] for so in speed_objects]
})

swimming_speed_df = swimming_speed_df.sort_values('captured_at', ascending=True)
swimming_speed_df.index = swimming_speed_df.captured_at
swimming_speed_df.index = pd.to_datetime(swimming_speed_df.index)


In [ ]:
tdf = swimming_speed_df.speed.resample('D').agg(lambda x: x.mean())


In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(swimming_speed_df.index, swimming_speed_df.speed)
tdf = swimming_speed_df.rolling('6H').mean().resample('H').agg(lambda x: x.mean())
plt.plot(tdf.index, tdf.values, color='red', linewidth=7)
plt.grid()
plt.show()

In [ ]:
tdf.index.date.astype(str)


In [ ]:
tdf.values

In [ ]:
pd.DataFrame({
    'date': tdf.index.date.astype(str),
    'speed': tdf.values
})